<a href="https://colab.research.google.com/github/JCaballerot/Recommender-Systems/blob/main/XGBoost_Recommender/Content_RecSys_BERT_y_XGBoost_Book_Crossing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
  <img src="https://storage.googleapis.com/kaggle-datasets-images/1661575/2726067/684ac0c4c14cb46d1047ccb620b45cac/dataset-cover.jpg?t=2021-10-21-03-18-09" width="800" height="300">
</center>


# **Content RecSys: BERT y XGBoost Book-Crossing**


---
## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item1">Introducción</a>  
2. <a href="#item4">Descripción del Dataset</a>  
3. <a href="#item4">Preprocesamiento de Datos</a>  
4. <a href="#item4">Muestreo/Enmascaramiento</a>  
5. <a href="#item4">Feature engineering</a>  
6. <a href="#item4">Preparación de los Datos para el Modelo</a>  
7. <a href="#item4">Entrenamiento del Modelo con XGBoost</a>  
8. <a href="#item4">Evaluación del Modelo</a>  
9. <a href="#item4">Generación de Recomendaciones</a>  
10. <a href="#item4">Conclusiones</a>  

</font>
</div>

---

## 1. Introducción

El dataset de **Book-Crossing** es un recurso rico en información sobre las interacciones de usuarios con libros, capturando calificaciones, títulos, autores y otros datos relevantes. Este laboratorio propone desarrollar un sistema de recomendación basado en **machine learning**, combinando técnicas avanzadas de procesamiento de texto con **BERT** y aprendizaje supervisado mediante **XGBoost**.

El objetivo principal es predecir si un usuario disfrutará un libro en particular, basado en sus interacciones pasadas y características tanto del usuario como del libro. Además, se busca generar recomendaciones personalizadas y evaluar métricas como la diversidad global de las recomendaciones.

A lo largo del laboratorio, se explorarán diferentes técnicas de ingeniería de características, como el uso de variables cuantitativas, target encoding para variables categóricas, y **embeddings** textuales obtenidos con **BERT**. Finalmente, se entrenará un modelo de **XGBoost**, y se evaluará su desempeño utilizando métricas como el coeficiente de Gini.

## 2. Descripción del Dataset

El dataset de Book-Crossing contiene 3 archivos principales:

- **BX-Users.csv:** Información sobre los usuarios.
- **BX-Books.csv:** Información sobre los libros.
- **BX-Book-Ratings.csv:** Calificaciones dadas por los usuarios a los libros.

El dataset puede ser descargado desde: Book-Crossing Dataset.



<a name="3.1"></a>

**2.1. BX-Users.csv**

Contiene información de los usuarios:

- **User-ID:** Identificador único del usuario.
- **Location:** Ubicación del usuario.
- **Age:** Edad del usuario.



<a name="3.2"></a>

**2.2. BX-Books.csv**

Contiene información de los libros:

- **SBN:** Identificador único del libro.
- **Book-Title:** Título del libro.
- **Book-Author:** Autor del libro.
- **Year-Of-Publication:** Año de publicación.
- **Publisher:** Editorial.


<a name="3.3"></a>

**2.3. BX-Book-Ratings.csv**
Contiene las calificaciones de los usuarios:

- **User-ID:** Identificador del usuario.
- **ISBN:** Identificador del libro.
- **Book-Rating:** Calificación dada al libro (0-10).


## 4. Preprocesamiento de Datos


En esta sección, cargaremos los datos y realizaremos una limpieza y exploración inicial.



**4.1. Carga de los Datos**

In [200]:

# Crear la carpeta llamada 'dataset'
!mkdir -p dataset

# Descargar los archivos CSV y guardarlos en la carpeta 'dataset'
!wget -O dataset/BX-Book-Ratings.csv https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Book-Ratings.csv
!wget -O dataset/BX-Books.csv https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Books.csv
!wget -O dataset/BX-Users.csv https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Users.csv


--2024-11-30 22:37:27--  https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Book-Ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29532460 (28M) [text/plain]
Saving to: ‘dataset/BX-Book-Ratings.csv’

dataset/BX-Book-Rat 100%[===================>]  28.16M  44.2MB/s    in 0.6s    

2024-11-30 22:37:28 (44.2 MB/s) - ‘dataset/BX-Book-Ratings.csv’ saved [29532460/29532460]

--2024-11-30 22:37:28--  https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Books.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP

In [201]:
# Importar las librerías necesarias
import pandas as pd

# Cargar los archivos CSV
ratings = pd.read_csv("dataset/BX-Book-Ratings.csv", sep=";", encoding="ISO-8859-1")
books = pd.read_csv("dataset/BX-Books.csv", sep=";", encoding="ISO-8859-1", on_bad_lines="skip", low_memory=False)
users = pd.read_csv("dataset/BX-Users.csv", sep=";", encoding="ISO-8859-1")

# Asignar manualmente los nombres de las columnas
ratings.columns = ['User-ID', 'ISBN', 'Book-Rating']
books.columns = ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
users.columns = ['User-ID', 'Location', 'Age']


**4.2. Exploración y Limpieza**

Visualizar las primeras filas de cada dataframe:

In [202]:
# Mostrar las primeras filas de ratings
print(ratings.head())

# Mostrar las primeras filas de books
print(books.head())

# Mostrar las primeras filas de users
print(users.head())


   User-ID        ISBN  Book-Rating
0   276726  0155061224            5
1   276727  0446520802            0
2   276729  052165615X            3
3   276729  0521795028            6
4   276733  2080674722            0
         ISBN                                         Book-Title  \
0  0002005018                                       Clara Callan   
1  0060973129                               Decision in Normandy   
2  0374157065  Flu: The Story of the Great Influenza Pandemic...   
3  0393045218                             The Mummies of Urumchi   
4  0399135782                             The Kitchen God's Wife   

            Book-Author Year-Of-Publication                   Publisher  \
0  Richard Bruce Wright                2001       HarperFlamingo Canada   
1          Carlo D'Este                1991             HarperPerennial   
2      Gina Bari Kolata                1999        Farrar Straus Giroux   
3       E. J. W. Barber                1999  W. W. Norton &amp; Company   


**Limpieza de Datos**

Conversión de tipos de datos.

Manejo de valores faltantes.

In [203]:
# Convertir 'Year-Of-Publication' a numérico y manejar errores
books['Year-Of-Publication'] = pd.to_numeric(books['Year-Of-Publication'], errors='coerce')
books['Year-Of-Publication'].fillna(books['Year-Of-Publication'].median(), inplace=True)
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype(int)


<ipython-input-203-7f58c4612ec2>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books['Year-Of-Publication'].fillna(books['Year-Of-Publication'].median(), inplace=True)


In [204]:
# Manejar valores faltantes en 'Book-Author' y 'Publisher'
books['Book-Author'].fillna('Unknown', inplace=True)
books['Publisher'].fillna('Unknown', inplace=True)


<ipython-input-204-71b7ce4f9bdd>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books['Book-Author'].fillna('Unknown', inplace=True)
<ipython-input-204-71b7ce4f9bdd>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [205]:
# Convertir 'Age' a numérico y manejar errores
users['Age'] = pd.to_numeric(users['Age'], errors='coerce')
users['Age'].fillna(users['Age'].median(), inplace=True)
users['Age'] = users['Age'].astype(int)

<ipython-input-205-9482be963f6e>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  users['Age'].fillna(users['Age'].median(), inplace=True)


## 5. División de los Datos

Para evaluar el modelo de manera adecuada, separaremos un 10% de las calificaciones de cada usuario para utilizarlas como conjunto de prueba.

In [206]:
from sklearn.model_selection import GroupShuffleSplit

# Unir ratings con usuarios y libros
data = ratings.merge(users, on='User-ID').merge(books, on='ISBN')[:10000]

# Binarizar el target: 1 si Book-Rating > 5, 0 si <= 5
data['target'] = (data['Book-Rating'] > 7).astype(int)


In [207]:
# Separar en entrenamiento y prueba por usuario
gss = GroupShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
train_idx, test_idx = next(gss.split(data, groups=data['User-ID']))

train_data = data.iloc[train_idx].reset_index(drop=True)
test_data = data.iloc[test_idx].reset_index(drop=True)


## 6. Feature Engineering

En esta sección, crearemos nuevas características que ayuden al modelo a predecir con mayor precisión.

**6.1. Variables Cuantitativas**

- Edad del usuario (Age).

- Antigüedad del libro: Año actual menos el año de publicación.

In [208]:
from datetime import datetime

current_year = datetime.now().year

# Calcular la antigüedad del libro
train_data['Book-Age'] = current_year - train_data['Year-Of-Publication']
test_data['Book-Age'] = current_year - test_data['Year-Of-Publication']


**6.2. Target Encoding**

Aplicaremos target encoding a las variables categóricas:

- Location
- Book-Author
- Publisher

In [209]:
%%capture
!pip3 install category_encoders

In [210]:
from category_encoders import TargetEncoder

categorical_features = ['Location', 'Book-Author', 'Publisher']

# Crear el encoder para las variables categóricas
encoder = TargetEncoder(cols=categorical_features)
encoder.fit(train_data[categorical_features], train_data['target'])

# Transformar las variables categóricas en los datos de entrenamiento y prueba
train_encoded = encoder.transform(train_data[categorical_features])
test_encoded = encoder.transform(test_data[categorical_features])

# Cambiar el nombre de las columnas codificadas
train_encoded.columns = [f"target_coded_{col}" for col in train_encoded.columns]
test_encoded.columns = [f"target_coded_{col}" for col in test_encoded.columns]

# Añadir las variables codificadas al dataframe original
train_data = pd.concat([train_data, train_encoded], axis=1)
test_data = pd.concat([test_data, test_encoded], axis=1)




**6.3. Generación de Embeddings con BERT**

Utilizaremos BERT para convertir el título del libro en embeddings numéricos que capturen su semántica.



**Instalación y Carga de BERT**

In [211]:
# Instalar la biblioteca transformers
!pip install transformers

from transformers import BertTokenizer, BertModel
import torch

# Cargar el tokenizador y el modelo preentrenado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


Función para Obtener Embeddings

In [212]:
def get_bert_embedding(text):
    # Tokenización y codificación
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=20)
    # Obtener las representaciones del modelo
    with torch.no_grad():
        outputs = model(**inputs)
    # Usar el embedding del token [CLS] como representación
    embedding = outputs.last_hidden_state[:,0,:].numpy()
    return embedding.flatten()


**Aplicar la Función a los Datos**

Debido a limitaciones computacionales, podemos trabajar con una muestra.

In [213]:
#%%time
#import numpy as np

# Obtener embeddings para el título del libro en el conjunto de entrenamiento
#train_embeddings = np.vstack(train_data['Book-Title'].apply(get_bert_embedding))

# Obtener embeddings para el conjunto de prueba
#test_embeddings = np.vstack(test_data['Book-Title'].apply(get_bert_embedding))


In [214]:
# Convierte los embeddings en un DataFrame
train_embeddings_df = pd.DataFrame(train_embeddings, columns=[f"Title_embedding_{i}" for i in range(train_embeddings.shape[1])])
train_data = pd.concat([train_data.reset_index(drop=True), train_embeddings_df.reset_index(drop=True)], axis=1)

# Convierte los embeddings en un DataFrame
test_embeddings_df = pd.DataFrame(test_embeddings, columns=[f"Title_embedding_{i}" for i in range(test_embeddings.shape[1])])
test_data = pd.concat([test_data.reset_index(drop=True), test_embeddings_df.reset_index(drop=True)], axis=1)


In [215]:
test_data.head()

,User-ID,ISBN,Book-Rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,...,Title_embedding_758,Title_embedding_759,Title_embedding_760,Title_embedding_761,Title_embedding_762,Title_embedding_763,Title_embedding_764,Title_embedding_765,Title_embedding_766,Title_embedding_767
0,276748,0747558167,6,"jubail ind.-city, eastern province, saudi arabia",39,Apricots on the Nile: A Memoir with Recipes,Colette Rossant,2002,Bloomsbury Publishing Plc,http://images.amazon.com/images/P/0747558167.0...,...,-0.196129,0.172129,-0.464506,0.322098,-0.025248,-0.106661,-0.210470,-0.277129,0.272023,0.505884
1,276833,0671876244,0,"fredericton, new brunswick, canada",14,"Field of Dishonor (Honor Harrington Series, Bo...",David Weber,1994,Baen,http://images.amazon.com/images/P/0671876244.0...,...,-0.373214,-0.056172,-0.026112,-0.200979,-0.020130,-0.288961,-0.459464,0.006871,0.347546,0.301249
2,276835,3125785006,10,"bitterfeld, sachsen-anhalt, germany",32,Of Mice and Men. Text and Study Aids. (Lernmat...,John Steinbeck,2001,Klett,http://images.amazon.com/images/P/3125785006.0...,...,0.129770,0.166401,-0.293951,-0.532792,0.066146,0.027168,-0.539871,-0.410446,0.151411,0.549408
3,276887,0064430227,5,"arlington, massachusetts, usa",32,Harold and the Purple Crayon 50th Anniversary ...,Crockett Johnson,1981,HarperTrophy,http://images.amazon.com/images/P/0064430227.0...,...,-0.512577,-0.011795,0.047334,-0.136785,0.590343,0.249061,-0.388317,-0.199091,-0.014575,0.680262
4,276887,0671723650,0,"arlington, massachusetts, usa",32,How To Win Friends And Influence People,Dale Carnegie,1990,Pocket,http://images.amazon.com/images/P/0671723650.0...,...,-0.270985,-0.116670,0.171218,-0.534190,-0.028804,0.379310,-0.233944,-0.352445,0.092769,0.356999


**6.4. One-Hot Encoding basado en Productos Calificados**

Crearemos características que indiquen si el usuario ya ha calificado el libro y el puntaje que le dio previamente.

**Crear un Diccionario de Calificaciones por Usuario**

In [216]:
data[['User-ID', 'ISBN', 'Book-Rating']]

,User-ID,ISBN,Book-Rating
0,276726,0155061224,5
1,276727,0446520802,0
2,276729,052165615X,3
3,276729,0521795028,6
4,276733,2080674722,0
...,...,...,...
9995,746,0812516826,5
9996,753,0679420118,10
9997,753,0741402858,8
9998,753,0741402912,10


In [217]:
# Realizar One-Hot Encoding sobre ISBN con los valores de 'Book-Rating'
one_hot_encoded = data.pivot_table(
    index='User-ID',
    columns='ISBN',
    values='Book-Rating')

# Ajustar los nombres de las columnas para incluir el prefijo 'book-rating-'
one_hot_encoded.columns = [f"book-rating-{isbn}" for isbn in one_hot_encoded.columns]
one_hot_encoded.head()

,book-rating-0002005018,book-rating-0002240114,book-rating-000225669X,book-rating-0002558122,book-rating-0002740230,book-rating-0006379702,book-rating-0006485294,book-rating-0006542808,book-rating-0006543545,book-rating-0006546684,...,book-rating-9508521481,book-rating-9681500555,book-rating-9681500830,book-rating-9722100718,book-rating-9722509713,book-rating-9724115380,book-rating-9724119378,book-rating-9726101794,book-rating-9871138016,book-rating-9995585227
User-ID,,,,,,,,,,,,,,,,,,,,,
8,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Aplicar la Función a los Datos**

In [218]:
# Asegúrate de que los índices sean consistentes
one_hot_encoded = one_hot_encoded.reset_index()  # Reseteamos el índice para usar 'User-ID' como columna

# Merge con train_data
train_data_merged = train_data.merge(one_hot_encoded, on='User-ID', how='left')

# Merge con test_data
test_data_merged = test_data.merge(one_hot_encoded, on='User-ID', how='left')


limpiamos los registros que coinciden con el target

In [219]:
# Para train_data
for idx, isbn in zip(train_data_merged.index, train_data_merged['ISBN']):
    column_to_nullify = f"book-rating-{isbn}"
    if column_to_nullify in train_data_merged.columns:
        train_data_merged.loc[idx, column_to_nullify] = np.nan

# Para test_data
for idx, isbn in zip(test_data_merged.index, test_data_merged['ISBN']):
    column_to_nullify = f"book-rating-{isbn}"
    if column_to_nullify in test_data_merged.columns:
        test_data_merged.loc[idx, column_to_nullify] = np.nan


## 7. Preparación de los Datos para el Modelo

Seleccionaremos las características y prepararemos los conjuntos de entrenamiento y prueba.

In [220]:
test_data_merged.columns.tolist()

['User-ID',
 'ISBN',
 'Book-Rating',
 'Location',
 'Age',
 'Book-Title',
 'Book-Author',
 'Year-Of-Publication',
 'Publisher',
 'Image-URL-S',
 'Image-URL-M',
 'Image-URL-L',
 'target',
 'Book-Age',
 'target_coded_Location',
 'target_coded_Book-Author',
 'target_coded_Publisher',
 'Title_embedding_0',
 'Title_embedding_1',
 'Title_embedding_2',
 'Title_embedding_3',
 'Title_embedding_4',
 'Title_embedding_5',
 'Title_embedding_6',
 'Title_embedding_7',
 'Title_embedding_8',
 'Title_embedding_9',
 'Title_embedding_10',
 'Title_embedding_11',
 'Title_embedding_12',
 'Title_embedding_13',
 'Title_embedding_14',
 'Title_embedding_15',
 'Title_embedding_16',
 'Title_embedding_17',
 'Title_embedding_18',
 'Title_embedding_19',
 'Title_embedding_20',
 'Title_embedding_21',
 'Title_embedding_22',
 'Title_embedding_23',
 'Title_embedding_24',
 'Title_embedding_25',
 'Title_embedding_26',
 'Title_embedding_27',
 'Title_embedding_28',
 'Title_embedding_29',
 'Title_embedding_30',
 'Title_embeddin

In [221]:
# Variables cuantitativas
features = ['Age', 'Book-Age'] + \
           [x for x in test_data_merged.columns if 'target_coded_' in x] +\
           [x for x in test_data_merged.columns if 'book-rating-' in x] +\
           [x for x in test_data_merged.columns if 'Title_embedding_' in x]

features = list(set(features))

In [222]:
# Combinar todas las características
X_train = train_data_merged[features]
X_test = test_data_merged[features]

# Variables objetivo
y_train = train_data_merged['target']
y_test = test_data_merged['target']


In [223]:
X_train.shape

(8842, 9852)

In [224]:
X_test.shape

(1158, 9852)

## 8. Entrenamiento del Modelo con XGBoost

Entrenaremos un modelo de clasificación utilizando XGBoost.

In [225]:
# Instalar XGBoost
!pip install xgboost

In [226]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dividir un conjunto de validación
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_val.shape

(1769, 9852)

In [227]:
# Crear las DMatrix para XGBoost
dtrain = xgb.DMatrix(X_train_split.values, label=y_train_split, feature_names=features)
dval   = xgb.DMatrix(X_val.values, label=y_val, feature_names=features)


In [228]:


# Configuración de los parámetros del modelo
params = {
    "objective": "binary:logistic",  # Cambiar si es un problema multiclase
    "max_depth": 6,
    "learning_rate": 0.01,
    "min_child_weight": 200,
    "subsample": 0.4,
    "colsample_bytree": 0.4,
    "eval_metric": "auc"
}

# Entrenar con early stopping
evals = [(dtrain, "train"), (dval, "eval")]
model = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    evals=evals,
    early_stopping_rounds=10,
    verbose_eval=True
)


[0]	train-auc:0.76824	eval-auc:0.76362
[1]	train-auc:0.79987	eval-auc:0.79768
[2]	train-auc:0.79937	eval-auc:0.79687
[3]	train-auc:0.80616	eval-auc:0.80617
[4]	train-auc:0.80646	eval-auc:0.80655
[5]	train-auc:0.80623	eval-auc:0.80616
[6]	train-auc:0.80646	eval-auc:0.80655
[7]	train-auc:0.80643	eval-auc:0.80646
[8]	train-auc:0.80643	eval-auc:0.80645
[9]	train-auc:0.80643	eval-auc:0.80644
[10]	train-auc:0.80643	eval-auc:0.80644
[11]	train-auc:0.80625	eval-auc:0.80621
[12]	train-auc:0.80643	eval-auc:0.80644
[13]	train-auc:0.80642	eval-auc:0.80639
[14]	train-auc:0.85385	eval-auc:0.84967
[15]	train-auc:0.85386	eval-auc:0.84970
[16]	train-auc:0.85418	eval-auc:0.84963
[17]	train-auc:0.85506	eval-auc:0.85430
[18]	train-auc:0.85519	eval-auc:0.85445
[19]	train-auc:0.85715	eval-auc:0.85560
[20]	train-auc:0.85982	eval-auc:0.85710
[21]	train-auc:0.86004	eval-auc:0.85743
[22]	train-auc:0.86116	eval-auc:0.86355
[23]	train-auc:0.86116	eval-auc:0.86082
[24]	train-auc:0.86118	eval-auc:0.86095
[25]	train

## 9. Evaluación del Modelo

Evaluaremos el rendimiento del modelo utilizando el coeficiente de Gini.

Evaluar el Modelo

In [232]:
from sklearn.metrics import roc_auc_score

# Predecir probabilidades en el conjunto de prueba
y_probs = model.predict(xgb.DMatrix(X_train_split))

# Calcular el Gini utilizando AUC
auc = roc_auc_score(y_train_split, y_probs)
auc*2-1

0.7245825448277459

In [233]:

# Predecir probabilidades en el conjunto de prueba
y_probs = model.predict(xgb.DMatrix(X_val))

# Calcular el Gini utilizando AUC
auc = roc_auc_score(y_val, y_probs)
auc*2-1

0.7144664750862608

## 10. Generación de Recomendaciones

Generaremos recomendaciones personalizadas para los usuarios y calcularemos la diversidad global.

**10.1. Predicción para Libros No Calificados**

Para cada usuario, recomendamos los 10 libros con mayor probabilidad de gustarle.

**10.2. Cálculo de la Diversidad Global**

Calculamos la proporción de libros únicos recomendados respecto al total del catálogo.

In [ ]:
# Conjunto de libros recomendados
recommended_books = set()

for recs in recommendations.values():
    recommended_books.update(recs)

diversity = len(recommended_books) / len(all_isbns)
print(f'Diversidad Global de Recomendaciones: {diversity:.4f}')


## 11. Conclusiones

En este laboratorio, hemos implementado un sistema de recomendación que combina técnicas avanzadas de procesamiento de lenguaje natural y aprendizaje automático:

**BERT** nos permitió convertir títulos de libros en representaciones numéricas que capturan su significado semántico.

**XGBoost** fue utilizado para entrenar un modelo de clasificación capaz de predecir si a un usuario le gustará un libro.

Mediante **feature engineerering**, incorporamos información relevante sobre usuarios y libros, mejorando la capacidad predictiva del modelo.

Las recomendaciones generadas ofrecen una diversidad global significativa, lo cual es beneficioso para mantener el interés del usuario y promover una exploración más amplia del catálogo.

Este enfoque demuestra cómo la integración de diferentes técnicas puede conducir a soluciones efectivas en el desarrollo de sistemas de recomendación personalizados.



---
## Gracias por completar este laboratorio!